In [1]:
# Impordid
from src.transformers.models.bert.tokenization_bert import BertTokenizer
from src.transformers.models.bert.modeling_bert import BertEmbeddings, BertModel, BertForMaskedLM
import torch
from estnltk import Text
from estnltk.corpus_processing.parse_enc import parse_enc_file_iterator
from src.transformers.models.bert.configuration_bert import BertConfig
from transformers import AdamW
from tqdm.auto import tqdm
from transformers import pipeline
import numpy as np

In [2]:
# Tokeniseerija
tokenizer = BertTokenizer(vocab_file = "vocab2.txt", vocab_file_form = "vocab_form.txt")

BertTokenizer


In [3]:
sisend = ["Väljas sajab lund.", "Mul ei tule und."]

In [4]:
# Dataseti loomine dataloaderi jaoks
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __len__(self):
        return self.encodings["input_ids"].shape[0]
    def __getitem__(self, i):
        return {key: tensor[i] for key, tensor in self.encodings.items()}

In [5]:
# Maskimine
def mlm(tensor):
    rand = torch.rand(tensor[:, :, 0].shape)
    mask_arr = (rand < 0.15) * (tensor[:, :, 0] > 5) # 15% sõnadest maskitakse
    for i in range(tensor[:, :, 0].shape[0]):
        selection = torch.flatten(mask_arr[i].nonzero()).tolist()
        tensor[i, selection] = 4 # [MASK] tokeni ID mõlemas vocabis
    return tensor

In [6]:
# Andmestiku ettevalmistamine treenimiseks või mudelis kasutamiseks
def prepare_data(data):
    input_ids = []
    mask = []
    labels = []
    
    sample = tokenizer(data, max_length = 8, padding = "max_length", truncation = True, return_tensors = "pt")
    labels.append(sample.input_ids[:, :, 1]) # Labeliks võtame sõnavormi id
    mask.append(sample.attention_mask)
    input_ids.append(mlm(sample.input_ids.detach().clone()))
    
    input_ids = torch.cat(input_ids)
    mask = torch.cat(mask)
    labels = torch.cat(labels)
    
    encodings = {
    "input_ids" : input_ids,
    "attention_mask" : mask,
    "labels" : labels
    }
    
    dataset = Dataset(encodings)
    return dataset


# Dataloaderi loomine mudeli treenimiseks
dataloader_train = torch.utils.data.DataLoader(prepare_data(sisend), batch_size = 16, shuffle = True)

In [7]:
%%time
# Mudeli treenimine

config = BertConfig(
    vocab_size = tokenizer.vocab_size,
    vocab_size_form = tokenizer.vocab_size_form,
    tie_word_embeddings = False
)

model = BertForMaskedLM(config)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

model.to(device)
optim = AdamW(model.parameters(), lr = 1e-4)
epochs = 3

for epoch in range(epochs):
    loop = tqdm(dataloader_train, leave = True)
    for batch in loop:
        optim.zero_grad()
        input_ids = batch["input_ids"].to(device)
        mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        outputs = model(input_ids, attention_mask = mask, labels = labels)
        loss = outputs.loss
        loss.backward()
        optim.step()

        loop.set_description(f"Epoch: {epoch}")
        loop.set_postfix(loss = loss.item())

C:\Users\rauln\anaconda3\envs\EKTP\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

CPU times: total: 28.4 s
Wall time: 5.31 s


In [ ]:
# Testimine

dataloader_test = torch.utils.data.DataLoader(prepare_data(test), batch_size = 16)

loop2 = tqdm(dataloader_test, leave = True)
for batch in loop2:
    input_ids = batch["input_ids"].to(device)
    outputs = model(input_ids)
    # Implementeerida täpsuse leidmine, kui muu kõik töötab

In [10]:
query = 'Mul ei tule [MASK]'
inp = tokenizer(query,return_tensors='pt')
mask_loc = np.where(inp.input_ids.numpy()[0] == 4)[0].tolist()[0]
out = model(inp["input_ids"]).logits[0].detach().numpy()
predicted_tokens = np.argmax(out[mask_loc]).tolist()
tokenizer.decode(predicted_tokens)
# tokenizer.decode praegu


0

In [12]:
tokenizer.added_tokens_decoder

{}

In [ ]:
model(inp["input_ids"]).logits[0]

In [ ]:
sisend = [id[0] for id in tokenizer('Tallinna [MASK] algatab Paldiski maantee ääres Hotell Tallinna kõrval asuva vundamendiaugu.').input_ids]
tokenizer.decode(sisend)
tokenizer("Tere mina olen keegi")

# Küsimused

1) Masked LM - õige lähenemine? jah
2) Kas peaks vocabi ka ise sisendi põhjal välja töötama? Ise arvan et jah, praegu päris paljud [UNK] teee oma sõnastik  jah

fikseerida tühi vorm
 - tokeniseerija võiks suvalise lemma võtta
lausealgus... mida ta sellega teeb? ega ta seda tavalise sõnana ei käsitle?
vormi ja lemmat korraga
softmax/CE - ennustus tõenäosusteks
samas treenida vormi enne ja lemmat pärast samas mudelis
loss = CE(V) + CE(L) see ümber kirjutada
- attention_mask, mis see on? V: sisuliselt [CLS] lause [SEP] saavad 1 ja [PAD] saab 0
teha notebook, mis teeb kogu asja ühe lause peal...
stack tracing - iga meetodi erroriga välja kutsuda, et teada kus see tuli

claudia kittask, meelis perli, ilmselt pole abi...
teises branchis stack tracimine...